Assignment - 2


Q1. Drinks Dataset

In [ ]:
import pandas as pd
drinkDf = pd.read_csv("drinks.csv")

Q1. a) Which continent drinks more wine on an average?

In [ ]:
t = drinkDf["continent"].unique()
npArr =  t[~pd.isnull(t)]
continents = []
for i in npArr :
    continents.append(i)
mean_arr = []
for i in continents :
    df = drinkDf[drinkDf['continent'].str.contains(i) == True]
    mean_arr.append(df['wine_servings'].mean())
df = drinkDf[drinkDf['continent'].isna()]
mean_arr.append(df['wine_servings'].mean())
continents.append('NA')
idx = mean_arr.index(max(mean_arr))
print(str(continents[idx]) + " drinks more wine on an average")

Q1. b) For each continent print the statistics for Beer consumption.

In [ ]:
drinkDf['continent'] = drinkDf['continent'].fillna('NA')
drinkDf.groupby('continent').beer_servings.describe()

Q1. c)  Print the median alcohol consumption per continent for every column.

In [ ]:
drinkDf['continent'] = drinkDf['continent'].fillna('NA')
drinkDf.groupby('continent').median()

Q2. Euro 2021 football tournament dataset

In [ ]:
Euro = pd.read_csv("Euro_2012_stats_TEAM.csv")

Q2. a) Find the teams who are participating in this tournament.

In [ ]:
Euro['Team']

Q2. b) Find the shooting accuracy of all the teams that have scored more than 5 goals.

In [ ]:
shooting_acc = Euro[Euro['Goals'] > 5]
shooting_acc[['Team','Goals','Shooting Accuracy']]

Q2. c) Find the total number of goals scored in the tournament.

In [ ]:
goals = Euro['Goals'].sum()
print(str(goals) + " scored in the tournament.")

Q2. d) Sort the teams on the basis of goals scored(non-ascending order)

In [ ]:
newDataFrame = Euro.copy()
copyFrame = newDataFrame.sort_values(by='Goals' , ascending=False)
copyFrame[['Team','Goals']]

Q2. e) Find the team with the most number of red cards.

In [ ]:
redCardNum = Euro['Red Cards']
teams = Euro['Team']
print(teams[redCardNum.idxmax()])

Q3. Tree Decision Classifier using numpy and pandas

In [ ]:
import numpy as np
import pandas as pd
treeDf = pd.read_csv("wdbc.csv" , header= None)
col = treeDf.columns
def swap(to , target):
    treeDf[[col[to], col[target]]] = treeDf[[col[target], col[to]]]
swap(1,31)


In [ ]:
class Vertex :
    def __init__(self, inx=None, ceiling=None, left=None, right=None, info_gain=None, value=None):
        self.inx , self.ceiling , self.left , self.right , self.info_gain , self.value = inx , ceiling , left , right , info_gain , value

In [ ]:
class DecisionTree():
    def __init__(self, splits=2, max_depth=2):
        self.start , self.splits , self.max_depth = None , splits , max_depth
    def buildTree(self, dataset, height=0):
        left, right = dataset[:,:-1], dataset[:,-1]
        getans, attributes = np.shape(left)
        if getans>=self.splits and height<=self.max_depth:
            best_split = self.BestGainSplit(dataset, attributes)
            if best_split["info_gain"]>0:
                l_child , r_child = self.buildTree(best_split["dataset_left"], height+1) , self.buildTree(best_split["dataset_right"], height+1)
                return Vertex(best_split["inx"], best_split["ceiling"], l_child, r_child, best_split["info_gain"])
        leafNodeVal = self.reached_end(right)
        return Vertex(value=leafNodeVal)
    def BestGainSplit(self, dataset, attributes):
        best_split , BestGainSoFar = {} , -float("inf")
        for inx in range(attributes):
            possibleVals  = dataset[:, inx] 
            cutoffVal =  np.unique(possibleVals)
            for ceiling in cutoffVal:
                dataset_left, dataset_right = self.split(dataset, inx, ceiling) #split the dataset at this CutoffValue
                if len(dataset_left)>0 and len(dataset_right)>0: #incase size is 0 for either we can't split here
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    GainATNode  = self.information_gain(y, left_y, right_y)
                    if GainATNode > BestGainSoFar:
                        best_split["inx"] , best_split["ceiling"] , best_split["dataset_left"] , best_split["dataset_right"] ,  best_split["info_gain"] , BestGainSoFar = inx , ceiling , dataset_left , dataset_right , GainATNode  , GainATNode 
        return best_split
    def split(self, dataset, inx, ceiling):
        return np.array([row for row in dataset if row[inx]<=ceiling]), np.array([row for row in dataset if row[inx]>ceiling])
    def information_gain(self, parent, l_child, r_child):  
        return self.getGini(parent) - ((len(l_child)/len(parent))*self.getGini(l_child) + (len(r_child)/len(parent))*self.getGini(r_child))
    def getGini(self, dataset):
        UniqueList , sizeOfData , sum =  np.unique(dataset) , len(dataset) , 0
        for probs in UniqueList :
            newGiniProb = len(dataset[dataset == probs])/sizeOfData
            sum += newGiniProb**2
        return 1 - sum
    def reached_end(self, df):  
        df = list(df)
        return max(df, key=df.count)  
    def train(self, X, Y):
        dataset = np.concatenate((X, Y), axis=1)
        self.start = self.buildTree(dataset)
    def AnswerByModel(self, X):
        Answers = [self.Traverse(x, self.start) for x in X]
        return Answers
    def Traverse(self, x, tree):
        if tree.value!=None: return tree.value
        if x[tree.inx]<=tree.ceiling: return self.Traverse(x, tree.left)
        else: return self.Traverse(x, tree.right)

In [ ]:
rows = treeDf.iloc[: , :-1].values
target = treeDf.iloc[:, -1].values.reshape(-1,1)


In [ ]:
from sklearn.model_selection import train_test_split
rows_train,rows_test , target_train, target_test = train_test_split(rows, target, test_size=.3, random_state=30)
rows_test

In [ ]:
Tree = DecisionTree(3,3)
Tree.train(rows_train,target_train)

In [ ]:
PredictionByModel = Tree.AnswerByModel(rows_test)
matched = 0
for i in range(len(PredictionByModel)) :
    if PredictionByModel[i] == target_test[i] :
        matched += 1
accuracy = matched/len(PredictionByModel)
print("Accuracy of the decision tree : " , str(accuracy))

Final Accuracy is 92.1875% on 1/3rd test tuples derived from main dataset.